# Регрессионный анализ: Kuiper.xls

- Корреляционный анализ; базовая линейная модель; выводы (коэффициенты, t-test, F-test);
- Диагностика: графики (scatter с линией регрессии, Residuals vs Fitted, Normal Q-Q, Residuals vs Leverage), проверка выбросов, influential observations (Cook's distance, leverage), VIF, тесты на гетероскедастичность, автокорреляцию, нормальность остатков;
- Stepwise (AIC) подбор модели и повторный анализ для улучшенной модели;
- Box–Cox трансформация отклика и анализ новой модели;
- Аналогичный анализ для датасета `cigarettes.txt` (модель: carbon monoxide ~ tar + nicotine + weight).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as sps

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import OLSInfluence, variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson

plt.rcParams['figure.figsize'] = (8,5)
plt.rcParams['font.size'] = 11
sns.set(style='whitegrid')


## Kuiper.xls — загрузка и предобработка

In [ ]:
kuiperDataFrame = pd.read_excel('../datasets/Kuiper.xls')
display(kuiperDataFrame.head())

def toFloatFromCommaSafe(x):
    try:
        if pd.isna(x):
            return np.nan
        s = str(x).strip().replace(',', '.')
        return float(s)
    except:
        return np.nan

numericCols = ['Price','Mileage','Cylinder','Liter','Cruise']
for col in numericCols:
    if col in kuiperDataFrame.columns:
        kuiperDataFrame[col] = kuiperDataFrame[col].apply(toFloatFromCommaSafe)
    else:
        print(f'Warning: column {col} not found in Kuiper.xls')

kuiperDataFrame = kuiperDataFrame.dropna(subset=['Price'])
kuiperDataFrame.describe()

### a) Корреляционный анализ
Построим матрицу корреляций по числовым переменным и тепловую карту.

In [ ]:
numDataKuiper = kuiperDataFrame[['Price','Mileage','Cylinder','Liter','Cruise']].dropna()
corrMatrixKuiper = numDataKuiper.corr()
print('Correlation matrix:')
display(corrMatrixKuiper)

sns.pairplot(numDataKuiper)
plt.suptitle('Pairplot — cigarettes', y=1.02)
plt.show()

plt.figure()
sns.heatmap(corrMatrixKuiper, annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Kuiper: Correlation matrix')
plt.show()


### b) Построение базовой модели линейной регрессии
Будем моделировать `Price` как функцию `Mileage + Cylinder + Liter + Cruise` (без категориального представления марки/модели).

In [ ]:
kuiperDataFrameClean = numDataKuiper.dropna().copy()
yKuiper = kuiperDataFrameClean['Price']
XKuiper = kuiperDataFrameClean[['Mileage','Cylinder','Liter','Cruise']]
XKuiperWithConst = sm.add_constant(XKuiper)

baseModelKuiper = sm.OLS(yKuiper, XKuiperWithConst).fit()
print('--- Base model summary ---')
print(baseModelKuiper.summary())


### c) Вывод результатов базовой модели
Стандартный `summary()` уже содержит коэффициенты, стандартные ошибки, t-statistics, p-values, R-squared и F-statistic. Ниже — явный вывод ключевых величин.

In [ ]:
coefficientsKuiper = baseModelKuiper.params
stdErrKuiper = baseModelKuiper.bse
tValuesKuiper = baseModelKuiper.tvalues
pValuesKuiper = baseModelKuiper.pvalues
fStatisticKuiper = baseModelKuiper.fvalue
fPvalueKuiper = baseModelKuiper.f_pvalue

print('\nCoefficients:\n', coefficientsKuiper, sep='')
print('\nStd. errors:\n', stdErrKuiper, sep='')
print('\nT-statistics:\n', tValuesKuiper, sep='')
print('\nP-values:\n', pValuesKuiper, sep='')
print('\nF-statistic = ', fStatisticKuiper, ', F p-value = ', fPvalueKuiper, sep='')


### d) Уравнение линейной регрессии
Запишем уравнение в явном виде (с коэффициентами из модели).

In [ ]:
interceptKuiper = coefficientsKuiper['const']
termsKuiper = []
for name in ['Mileage','Cylinder','Liter','Cruise']:
    coef = coefficientsKuiper[name]
    termsKuiper.append(f"({coef:.4f})*{name}")
equationKuiper = f"Price = {interceptKuiper:.4f} + " + ' + '.join(termsKuiper)
print('Regression equation:')
print(equationKuiper)


### e) T-test значимости коэффициентов
Статистики t и p-values для каждого коэффициента.

In [ ]:
# Тест всех коэффициентов (H0: каждый коэффициент = 0)
identityMatrix = np.eye(len(baseModelKuiper.params))
ttestAll = baseModelKuiper.t_test(identityMatrix)
print(ttestAll.summary())

### f) F-test значимости уравнения регрессии
F-statistic и его p-value.

In [ ]:
fStat = baseModelKuiper.fvalue
fPvalue = baseModelKuiper.f_pvalue
dfModel = int(baseModelKuiper.df_model) 
dfResid = int(baseModelKuiper.df_resid)
print(f'F-статистика = {fStat:.6f}')
print(f'p-значение для F = {fPvalue:.6f}')
print(f'df_model = {dfModel}, df_resid = {dfResid}')

### g) Scatterplots и линию регрессии
Построим scatter `Price` против каждого предиктора и линию предсказаний (прочие предикторы зафиксированы на средних).

In [ ]:
for xVar in ['Mileage', 'Cylinder', 'Liter', 'Cruise']:
    plt.figure()
    plt.scatter(kuiperDataFrameClean[xVar], yKuiper, alpha=0.6)

    xGrid = np.linspace(kuiperDataFrameClean[xVar].min(), kuiperDataFrameClean[xVar].max(), 100)
    meanValues = XKuiper.mean()

    predDf = pd.DataFrame({
        'Mileage': np.full_like(xGrid, meanValues['Mileage']),
        'Cylinder': np.full_like(xGrid, meanValues['Cylinder']),
        'Liter': np.full_like(xGrid, meanValues['Liter']),
        'Cruise': np.full_like(xGrid, meanValues['Cruise'])
    })
    predDf[xVar] = xGrid

    predDf = sm.add_constant(predDf, has_constant='add')
    yPredLine = baseModelKuiper.predict(predDf[baseModelKuiper.params.index])

    plt.plot(xGrid, yPredLine, color='red')
    plt.xlabel(xVar)
    plt.ylabel('Price')
    plt.title(f'Price vs {xVar} with regression line (others at mean)')
    plt.show()

### h) Доверительные интервалы для коэффициентов регрессии
Используем `conf_int()` из результатов модели.

In [ ]:
ciKuiper = baseModelKuiper.conf_int(alpha=0.05)
ciKuiper.columns = ['CI_lower','CI_upper']
print('95% confidence intervals for coefficients:')
display(ciKuiper)

### i) Важные наблюдения — влияние (influential observations)
Посчитаем Cook's distance и leverage; выведем великие наблюдения.

In [ ]:
influenceKuiper = OLSInfluence(baseModelKuiper)
cooksD = influenceKuiper.cooks_distance[0]
leverage = influenceKuiper.hat_matrix_diag
standardizedResiduals = influenceKuiper.resid_studentized_internal

influenceSummaryDf = kuiperDataFrameClean.copy()
influenceSummaryDf['cooksD'] = cooksD
influenceSummaryDf['leverage'] = leverage
influenceSummaryDf['stdResid'] = standardizedResiduals

topCooks = influenceSummaryDf.sort_values('cooksD', ascending=False).head(10)
print('Top observations by Cook''s distance:')
display(topCooks[['Price','Mileage','Cylinder','Liter','Cruise','cooksD','leverage','stdResid']])


### j) Stepwise selection по AIC (forward-backward)
Реализуем простую функцию stepwise_selection, которая выбирает переменные по минимальному AIC.

In [ ]:
def stepwiseSelection(X, y, initialList=[], thresholdIn=0.01, thresholdOut=0.05, verbose=True):
    included = list(initialList)
    while True:
        changed=False
        excluded = list(set(X.columns)-set(included))
        bestAic = None
        bestToAdd = None

        for newCol in excluded:
            tryCols = included + [newCol]
            model = sm.OLS(y, sm.add_constant(X[tryCols])).fit()
            aic = model.aic
            if bestAic is None or aic < bestAic:
                bestAic = aic
                bestToAdd = newCol

        if bestToAdd is not None:
            currentModel = sm.OLS(y, sm.add_constant(X[included]) ).fit() if included else None
            currentAic = currentModel.aic if currentModel is not None else np.inf
            if bestAic + 1e-8 < currentAic:
                included.append(bestToAdd)
                changed=True
                if verbose:
                    print('Add  {:30} with AIC {:.6f}'.format(bestToAdd, bestAic))

        if included:
            bestAic = None
            worstToRemove = None
            for col in included:
                tryCols = list(included)
                tryCols.remove(col)
                model = sm.OLS(y, sm.add_constant(X[tryCols]) ).fit()
                aic = model.aic
                if bestAic is None or aic < bestAic:
                    bestAic = aic
                    worstToRemove = col
            currentModel = sm.OLS(y, sm.add_constant(X[included]) ).fit()
            currentAic = currentModel.aic
            if bestAic + 1e-8 < currentAic:
                included.remove(worstToRemove)
                changed=True
                if verbose:
                    print('Remove {:30} to improve AIC to {:.6f}'.format(worstToRemove, bestAic))

        if not changed:
            break
    
    return included

candidateX = XKuiper.copy()
selectedVarsKuiper = stepwiseSelection(candidateX, yKuiper, verbose=True)
print('\nSelected variables by stepwise AIC:', selectedVarsKuiper)


### k) Если модель улучшилась — повторим пункты c–i для новой модели
Если `selectedVarsKuiper` отличается от исходного набора, подгоним новую модель и повторим диагностику.

In [ ]:
if set(selectedVarsKuiper) != set(XKuiper.columns):
    print('Fitting improved model with vars:', selectedVarsKuiper)
    XKuiperSelected = sm.add_constant(XKuiper[selectedVarsKuiper])
    improvedModelKuiper = sm.OLS(yKuiper, XKuiperSelected).fit()
    print(improvedModelKuiper.summary())
else:
    print('Stepwise did not change the model (selected == full).')


### l) Построить diagnostic-графики и дать интерпретации
Построим: scatterplot (все пары), Residuals vs Fitted, Normal Q-Q, Residuals vs Leverage (с указанием Cook's distance).

In [ ]:
sns.pairplot(kuiperDataFrameClean[['Price','Mileage','Cylinder','Liter','Cruise']])
plt.suptitle('Pairplot (Kuiper)', y=1.02)
plt.show()

# Residuals vs Fitted
fittedVals = baseModelKuiper.fittedvalues
residuals = baseModelKuiper.resid
plt.figure()
plt.scatter(fittedVals, residuals, alpha=0.6)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted')
plt.show()

# Normal Q-Q
sm.qqplot(residuals, line='45', fit=True)
plt.title('Normal Q-Q')
plt.show()

# Residuals vs Leverage (with Cook's distance contour)
fig = plt.figure(figsize=(8,6))
sm.graphics.influence_plot(baseModelKuiper, criterion='cooks')
plt.title('Residuals vs Leverage (influence plot)')
plt.show()


### m) Проверка на выбросы (outliers)
- Посмотреть большие по абсолютному значению стандартизованные/студентализованные остатки (|t_resid| > 3).

In [ ]:
influence = OLSInfluence(baseModelKuiper)
studentResid = influence.resid_studentized_external
outlierMask = np.abs(studentResid) > 3
print('Number of potential outliers (|studentized resid| > 3):', outlierMask.sum())
if outlierMask.sum()>0:
    display(kuiperDataFrameClean.loc[outlierMask, ['Price','Mileage','Cylinder','Liter','Cruise']].assign(studentResid=studentResid[outlierMask]))

### n) Тест на гетероскедастичность
- Breusch–Pagan тест.

In [ ]:
bpTest = het_breuschpagan(residuals, baseModelKuiper.model.exog)
bpStat, bpPvalue = bpTest[0], bpTest[1]
print('\nBreusch-Pagan: statistic =', bpStat, ', p-value =', bpPvalue)

### o) Тест на автокорреляцию остатков
- Durbin–Watson.

In [ ]:
dwStat = durbin_watson(residuals)
print('\nDurbin-Watson statistic =', dwStat)

### p) Нормальность остатков
- Shapiro и Jarque–Bera.

In [ ]:
jbStat, jbPvalue, skewResid, kurtResid = sm.stats.stattools.jarque_bera(residuals)
shStatRes, shPRes = sps.shapiro(residuals)
print('\nJarque-Bera: stat =', jbStat, ', p-value =', jbPvalue)
print('Shapiro-Wilk on residuals: stat =', shStatRes, ', p-value =', shPRes)

### q) Мультиколлинеарность
- VIF для каждой регрессора.

In [ ]:
vifData = pd.DataFrame()
vifData['feature'] = XKuiper.columns
vifData['VIF'] = [variance_inflation_factor(XKuiper.values, i) for i in range(XKuiper.shape[1])]
print('\nVIF:')
display(vifData)

### r) Попытка Box–Cox трансформации отклика `Price`
### s) Если Box–Cox дал новую модель — проанализировать её.  

In [ ]:
yBox, lambdaBox = sps.boxcox(yKuiper)
print('Box-Cox lambda =', lambdaBox)

modelBoxKuiper = sm.OLS(yBox, XKuiperWithConst).fit()
print(modelBoxKuiper.summary())
print('\nBase AIC =', baseModelKuiper.aic, ', Box-Cox model AIC =', modelBoxKuiper.aic)

# Регрессионный анализ: cigarettes.dat.txt

Анализ включает полный набор шагов, аналогичный ноутбуку для `Kuiper.xls`:
- загрузка и предобработка данных;
- корреляционный анализ и EDA (гистограммы/парные графики);
- построение базовой линейной модели: `carbonMonoxide ~ tar + nicotine + weight`;
- вывод результата (коэффициенты, t-test для каждого коэффициента, F-test для модели);
- уравнение регрессии;
- доверительные интервалы для коэффициентов;
- влияние (Cook's distance, leverage) и поиск influential observations;
- пошаговый подбор модели по AIC (stepwise);
- diagnostic-графики: Residuals vs Fitted, Normal Q-Q, Residuals vs Leverage;
- тесты: Breusch–Pagan (гетероскедастичность), Durbin–Watson (автокорреляция), Jarque–Bera и Shapiro (нормальность остатков), VIF (мультиколлинеарность);
- Box–Cox трансформация отклика и повторный анализ.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as sps

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import OLSInfluence, variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson

plt.rcParams['figure.figsize'] = (8,5)
plt.rcParams['font.size'] = 11
sns.set(style='whitegrid')


## Загрузка и предобработка `cigarettes.dat.txt`

In [ ]:
rawLines = []
with open('../datasets/cigarettes.dat.txt', 'r', encoding='utf-8') as fileHandle:
    for line in fileHandle:
        lineStr = line.strip()
        if lineStr:
            rawLines.append(lineStr)

rows = []
for line in rawLines:
    tokens = line.split()
    if len(tokens) < 5:
        continue
    numericTokens = tokens[-4:]
    brandTokens = tokens[:-4]
    brand = ' '.join(brandTokens)
    try:
        carbonMonoxideVal = float(numericTokens[0].replace(',', '.'))
        tarVal = float(numericTokens[1].replace(',', '.'))
        nicotineVal = float(numericTokens[2].replace(',', '.'))
        weightVal = float(numericTokens[3].replace(',', '.'))
    except Exception as parseErr:
        continue
    rows.append((brand, carbonMonoxideVal, tarVal, nicotineVal, weightVal))

cigarettesDataFrame = pd.DataFrame(rows, columns=['brand','carbonMonoxide','tar','nicotine','weight'])
print('Loaded rows =', len(cigarettesDataFrame))
display(cigarettesDataFrame.head(10))
print('\nSummary:')
display(cigarettesDataFrame.describe())


### a) Корреляционный анализ

In [ ]:
numericCig = cigarettesDataFrame[['carbonMonoxide','tar','nicotine','weight']].dropna()
print('Correlation matrix:')
display(numericCig.corr())

sns.pairplot(numericCig)
plt.suptitle('Pairplot — cigarettes', y=1.02)
plt.show()

plt.figure()
sns.heatmap(numericCig.corr(), annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Correlation matrix — cigarettes')
plt.show()


### b) Базовая линейная модель
Задаём модель: `carbonMonoxide ~ tar + nicotine + weight`

In [ ]:
cigClean = cigarettesDataFrame.dropna(subset=['carbonMonoxide','tar','nicotine','weight']).copy()
yCig = cigClean['carbonMonoxide']
XCig = cigClean[['tar','nicotine','weight']]
XCigWithConst = sm.add_constant(XCig)
cigModel = sm.OLS(yCig, XCigWithConst).fit()
print('--- Base OLS model summary ---')
print(cigModel.summary())


### c) Вывод результатов базовой модели
Стандартный `summary()` уже содержит коэффициенты, стандартные ошибки, t-statistics, p-values, R-squared и F-statistic. Ниже — явный вывод ключевых величин.

In [ ]:
coefficientsCig = cigModel.params
stdErrCig = cigModel.bse
tValuesCig = cigModel.tvalues
pValuesCig = cigModel.pvalues
fStatisticCig = cigModel.fvalue
fPvalueCig = cigModel.f_pvalue

print('\nCoefficients:\n', coefficientsCig, sep='')
print('\nStd. errors:\n', stdErrCig, sep='')
print('\nT-statistics:\n', tValuesCig, sep='')
print('\nP-values:\n', pValuesCig, sep='')
print('\nF-statistic = ', fStatisticCig, ', F p-value = ', fPvalueCig, sep='')


### d) Уравнение линейной регрессии
Запишем уравнение в явном виде (с коэффициентами из модели).

In [ ]:
interceptCig = coefficientsCig['const']
termsCig = []
for name in ['tar','nicotine','weight']:
    termsCig.append(f"({coefficientsCig[name]:.4f})*{name}")
equationCig = f"carbonMonoxide = {interceptCig:.4f} + " + ' + '.join(termsCig)
print('Regression equation:')
print(equationCig)


### e) T-test значимости коэффициентов
Статистики t и p-values для каждого коэффициента.

In [ ]:
# Тест всех коэффициентов (H0: каждый коэффициент = 0)
identityMatrix = np.eye(len(cigModel.params))
tTestAll = cigModel.t_test(identityMatrix)
print(tTestAll.summary())

### f) F-test значимости уравнения регрессии
F-statistic и его p-value.

In [ ]:
fStat = cigModel.fvalue
fP = cigModel.f_pvalue
print(f'F-statistic = {fStat:.6f}, p-value = {fP:.6f}')


### g) Scatterplots и линию регрессии
Построим scatter `carbonMonoxide` против каждого предиктора и линию предсказаний (прочие предикторы зафиксированы на средних).

In [ ]:
for xVar in ['tar', 'nicotine', 'weight']:
    plt.figure()
    plt.scatter(cigClean[xVar], yCig, alpha=0.7)

    xGrid = np.linspace(cigClean[xVar].min(), cigClean[xVar].max(), 100)
    meanValues = XCig.mean()

    predDf = pd.DataFrame({
        'tar': np.full_like(xGrid, meanValues['tar']),
        'nicotine': np.full_like(xGrid, meanValues['nicotine']),
        'weight': np.full_like(xGrid, meanValues['weight'])
    })
    predDf[xVar] = xGrid

    predDf = sm.add_constant(predDf, has_constant='add')
    yPred = cigModel.predict(predDf[cigModel.params.index])

    plt.plot(xGrid, yPred, color='red')
    plt.xlabel(xVar)
    plt.ylabel('carbonMonoxide')
    plt.title(f'carbonMonoxide vs {xVar} with regression line')
    plt.show()


### h) Доверительные интервалы для коэффициентов регрессии
Используем `conf_int()` из результатов модели.

In [ ]:
ciCig = cigModel.conf_int(alpha=0.05)
ciCig.columns = ['CI_lower','CI_upper']
print('95% confidence intervals for coefficients:')
display(ciCig)


### i) Важные наблюдения — влияние (influential observations)
Посчитаем Cook's distance и leverage; выведем великие наблюдения.

In [ ]:
influenceCig = OLSInfluence(cigModel)
cooksDCig = influenceCig.cooks_distance[0]
leverageCig = influenceCig.hat_matrix_diag
studentResidCig = influenceCig.resid_studentized_external

influenceSummaryCig = cigClean.copy()
influenceSummaryCig['cooksD'] = cooksDCig
influenceSummaryCig['leverage'] = leverageCig
influenceSummaryCig['studentResid'] = studentResidCig

print('Top observations by Cook''s distance:')
display(influenceSummaryCig.sort_values('cooksD', ascending=False).head(10))


### j) Stepwise selection по AIC (forward-backward)
Реализуем простую функцию stepwise_selection, которая выбирает переменные по минимальному AIC.

In [ ]:
def stepwiseSelection(X, y, initialList=[], verbose=True):
    included = list(initialList)
    while True:
        changed=False
        excluded = list(set(X.columns)-set(included))
        bestAic = None
        bestToAdd = None
        for newCol in excluded:
            tryCols = included + [newCol]
            model = sm.OLS(y, sm.add_constant(X[tryCols])).fit()
            aic = model.aic
            if bestAic is None or aic < bestAic:
                bestAic = aic
                bestToAdd = newCol
        if bestToAdd is not None:
            currentModel = sm.OLS(y, sm.add_constant(X[included]) ).fit() if included else None
            currentAic = currentModel.aic if currentModel is not None else np.inf
            if bestAic + 1e-8 < currentAic:
                included.append(bestToAdd)
                changed=True
                if verbose:
                    print('Add {:20} with AIC {:.6f}'.format(bestToAdd, bestAic))
        if included:
            bestAic = None
            worstToRemove = None
            for col in included:
                tryCols = list(included)
                tryCols.remove(col)
                model = sm.OLS(y, sm.add_constant(X[tryCols]) ).fit()
                aic = model.aic
                if bestAic is None or aic < bestAic:
                    bestAic = aic
                    worstToRemove = col
            currentModel = sm.OLS(y, sm.add_constant(X[included]) ).fit()
            currentAic = currentModel.aic
            if bestAic + 1e-8 < currentAic:
                included.remove(worstToRemove)
                changed=True
                if verbose:
                    print('Remove {:20} to improve AIC to {:.6f}'.format(worstToRemove, bestAic))
        if not changed:
            break
    return included

selectedVarsCig = stepwiseSelection(XCig, yCig, verbose=True)
print('\nSelected variables by stepwise AIC:', selectedVarsCig)


### k) Если модель улучшилась — повторим пункты c–i для новой модели
Если `selectedVarsCig` отличается от исходного набора, подгоним новую модель и повторим диагностику.

In [ ]:
if set(selectedVarsCig) != set(XCig.columns):
    print('Fitting improved model with vars:', selectedVarsCig)
    XCigSelected = sm.add_constant(XCig[selectedVarsCig])
    improvedCigModel = sm.OLS(yCig, XCigSelected).fit()
    print(improvedCigModel.summary())
else:
    print('Stepwise did not change the model (selected == full).')


### l) Построить diagnostic-графики и дать интерпретации
Построим: scatterplot (все пары), Residuals vs Fitted, Normal Q-Q, Residuals vs Leverage (с указанием Cook's distance).

In [ ]:
sns.pairplot(cigClean[['carbonMonoxide','tar','nicotine','weight']])
plt.suptitle('Pairplot (Kuiper)', y=1.02)
plt.show()

# Residuals vs Fitted
plt.figure()
plt.scatter(cigModel.fittedvalues, cigModel.resid, alpha=0.7)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted (cigModel)')
plt.show()

# Normal Q-Q
sm.qqplot(cigModel.resid, line='45', fit=True)
plt.title('Normal Q-Q (cigModel residuals)')
plt.show()

# Residuals vs Leverage (influence)
fig = plt.figure(figsize=(8,6))
sm.graphics.influence_plot(cigModel, criterion='cooks')
plt.title('Residuals vs Leverage (cigModel)')
plt.show()


### m) Проверка на выбросы (outliers)
- Посмотреть большие по абсолютному значению стандартизованные/студентализованные остатки (|t_resid| > 3).

In [ ]:
influence = OLSInfluence(cigModel)
studentResid = influence.resid_studentized_external
outlierMask = np.abs(studentResid) > 3
print('Number of potential outliers (|studentized resid| > 3):', outlierMask.sum())
if outlierMask.sum()>0:
    display(cigClean.loc[outlierMask, ['brand','carbonMonoxide','tar','nicotine','weight']].assign(studentResid=studentResid[outlierMask]))

### n) Тест на гетероскедастичность
- Breusch–Pagan тест.

In [ ]:
bpTest = het_breuschpagan(cigModel.resid, cigModel.model.exog)
print('\nBreusch-Pagan: LM stat =', bpTest[0], ', p-value =', bpTest[1])

### o) Тест на автокорреляцию остатков
- Durbin–Watson.

In [ ]:
dw = durbin_watson(cigModel.resid)
print('\nDurbin-Watson statistic =', dw)

### p) Нормальность остатков
- Shapiro и Jarque–Bera.

In [ ]:
jbStat, jbP, skewResid, kurtResid = sm.stats.stattools.jarque_bera(cigModel.resid)
shStat, shP = sps.shapiro(cigModel.resid)
print('\nJarque-Bera: stat =', jbStat, ', p-value =', jbP)
print('Shapiro-Wilk: stat =', shStat, ', p-value =', shP)

### q) Мультиколлинеарность
- VIF для каждой регрессора.

In [ ]:
vifCig = pd.DataFrame()
vifCig['feature'] = XCig.columns
vifCig['VIF'] = [variance_inflation_factor(XCig.values, i) for i in range(XCig.shape[1])]
print('\nVIF:')
display(vifCig)


### r) Box–Cox трансформация отклика
### s) Если Box–Cox дал новую модель — проанализировать её.  

In [ ]:
yCigBox, lambdaCig = sps.boxcox(yCig)
print('Box-Cox lambda for carbonMonoxide =', lambdaCig)

modelCigBox = sm.OLS(yCigBox, XCigWithConst).fit()
print('\nBox-Cox model summary:')
print(modelCigBox.summary())
print('\nOriginal AIC =', cigModel.aic, ', Box-Cox AIC =', modelCigBox.aic)